In [2]:
from lasso.dyna.D3plot import D3plot
from lasso.dyna.ArrayType import ArrayType
from lasso.dimred.sphere import *

from typing import List, Dict, Union, Tuple, Set

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from ipywidgets import Image, Layout

import math
import numpy as np
import ntpath
import glob
import re

In [3]:
filepath_list = glob.glob("exercise-1/d3plots/*.d3plot")
print(f"Found {len(filepath_list)} files.")

Found 100 files.


In [4]:
# we select the last timestep
last_timestep = -1

# let's collect node displacement from all d3plots
fields = []

for i_filepath, filepath in enumerate(filepath_list):
    
    # print progress
    print(f"filepath {filepath} ({i_filepath + 1}/{len(filepath_list)})", end='\r')
    
    # load d3plot
    d3plot = D3plot(filepath)
    
    # extract node displacement at last timestep
    node_displacement = d3plot.arrays[ArrayType.node_displacement]
    fields.append(node_displacement[last_timestep])

# print end
print(f"Done ({i_filepath + 1}/{len(filepath_list)})")

Done (100/100)ise-1/d3plots\run99.d3plot (100/100)


In [13]:
# load base model coordinates
plt = D3plot(filepath_list[0])
node_coords = plt.arrays["node_coordinates"]
print(node_coords.shape)

(2500, 3)


In [14]:
# convert list into a numpy array
data = np.stack(fields)

print("(n_simulations, n_nodes, xyz)")
print(data.shape)

(n_simulations, n_nodes, xyz)
(100, 2500, 3)


In [7]:
# load images
image_data = {}
for img_filepath in glob.glob('exercise-1/images/*.png'):
    with open(img_filepath, "rb") as f:
        print(img_filepath, end='\r')
        number = ntpath.basename(img_filepath).split('.')[0]
        image_data[int(number)] = f.read()
print()

exercise-1/images\99.png


In [8]:
# create embedding space where we store the projected results
histo = create_historgram(node_coords,'X',True)


In [9]:
n_models = len(data)

# perform hashing
# where to store all results
hashes = np.empty((n_models, 144*144), dtype=float)

for ii in range(n_models):
    # compute distance
    diff = data[ii] - node_coords[np.newaxis, :, :]
    
    # for this embedding we choose to use the node displacment
    # as a field value, one can also use the element p_strain
    # or stresses
    # we take the l2-norm of the diff so that we have
    # one scalar value we can map to a point
    part_fields = np.linalg.norm(diff, axis=2)

    # map the field values to the embedding space
    hashes[ii] = sphere_hashing(
        histo, part_fields[-1, :])


In [10]:
# perform mds
smatrix = compute_similarity(hashes)

mds = MDS(n_components=3,
          metric=True,
          n_init=4,
          max_iter=3000,
          verbose=0,
          eps=1e-5,
          dissimilarity='precomputed',
          n_jobs=4, )

# dissimilatiry hence we subtract from one
pos = mds.fit(1-smatrix).embedding_

In [11]:
# plot stuff and mds interaction
image_widget = Image(
    value=image_data[0],
    layout=Layout(height='500px', width='375px')
)

# plot
fig = go.FigureWidget(
    data=[
        dict(
            type='scatter3d',
            x=pos[:, 0],
            y=pos[:, 1],
            z=pos[:, 2],
            mode='markers',
            marker={
               "size": 3
            },
            opacity=.80,
            text=filepath_list,
            hoverinfo="text",
        )
    ],
)
scatter = fig.data[0]
scatter


def hover_fn(trace, points, state):
    ind = points.point_inds[0]
    filepath = filepath_list[ind]
    name = ntpath.basename(filepath).split('.')[0]
    number = int(re.search(r'\d+', name).group())

    # Update image widget
    image_widget.value = image_data[number]

scatter.on_hover(hover_fn)



In [15]:
from ipywidgets import HBox, VBox
HBox([fig,
      image_widget])

    'data': [{'hoverinfo': 'text',
              'marker': {'size': 3},
        …